In [ ]:
import os
import cv2
import keras
import pickle

In [ ]:
INT_S1_DIR = "_intermediate/stage1"
INT_S2_DIR = "_intermediate/stage2"

In [ ]:
os.makedirs(INT_S2_DIR, exist_ok=True)

In [ ]:
def get_latest_checkpoint(dir):
    if not os.path.isdir(dir):
        return None
    ckpts = [file for file in os.listdir(dir) if file.endswith("keras")]
    ckpts.sort()
    return os.path.join(dir, ckpts[-1])

In [ ]:
with open(os.path.join(INT_DIR, "total_data.pkl"), 'rb') as file:
    stage1_results = pickle.load(file, protocol)

In [ ]:
stage2_names = ["InceptionV3", "VGG19", "MobileNetV3Large"]

In [ ]:
# Load the latest checkpoints
stage2_dict = {
    name: keras.models.load_model(get_latest_checkpoint(f"../disease_detection/checkpoints/{name}_rgb")) 
    for name in stage2_names 
    if get_latest_checkpoint(f"../disease_detection/checkpoints/{name}_rgb") is not None
}

In [ ]:
# Load best performing model by type
stage2_dict = {
    name: keras.models.load_model(f"../disease_detection/out/best{name}_rgb")
    for name in stage2_names 
}

In [ ]:
stage2_results = {}
for stage2_name, stage2_model in stage2_dict.items():
    for stage1_name, stage1_result in stage1_results.items():
        for index, data in stage1_result.items():
            healthy = data['healthy']
            masks = data['masks']
            data['predictions'] = []
            for mask in masks:
                patch = mask['patch']
                inputs = cv2.resize(patch, (224, 224))
                inputs = tf.constant(inputs)
                inputs = tf.expand_dims(inputs, 0)
                prob = stage2_model(inputs)
                data['predictions'].append(prob)
                mask['prob_diseased'] = prob

In [ ]:
stage2_results = {}
for stage1_name, stage1_result in stage1_results.items():
    for inputs in stage1_result:
        for stage2_name, stage2_model in stage2_dict.items():
            stage2_results[f"{stage1_name} + {stage2_name}"] = []
            inputs = cv2.resize(inputs, (224, 224))
            inputs = tf.constant(inputs)
            inputs = tf.expand_dims(inputs, 0)
            prob = stage2_model(inputs)
            if prob > PROB_THRESHOLD:
                stage2_results[f"{stage1_name} + {stage2_name}"].append(index)
                #results.append(index)
                print(f"index {index} sick")